In [20]:
import tensorflow as tf
import os
import cv2
import pandas as pd
import numpy as np

In [21]:
# 이미지 데이터와 라벨을 저장할 리스트 초기화
data = []
labels = []

def make_dataset(dir_path, label):
    image_files = [f for f in os.listdir(dir_path) if f.endswith('.png')]
    
    # 이미지 파일 반복 처리
    for image_file in image_files:
        # 이미지 파일 경로 생성
        image_path = os.path.join(dir_path, image_file)
        
        # 이미지 읽기 (OpenCV 사용)
        image = cv2.imread(image_path)
        
        # 이미지 크기 조정 (필요한 경우)
        # image = cv2.resize(image, (width, height))
        
        # 이미지 데이터와 라벨 추가
        data.append(image)
        labels.append(label)  # 클래스 라벨을 적절하게 설정해야 합니다.



In [22]:
data_dir_0 = "C:/ssafy/candle/0/"
data_dir_1 = "C:/ssafy/candle/1/"

make_dataset(data_dir_0, 0)
make_dataset(data_dir_1, 1)

# 데이터와 라벨을 NumPy 배열로 변환
data = np.array(data)
labels = np.array(labels)

# 라벨을 원-핫 인코딩 (필요한 경우)
# labels = to_categorical(labels)
print(data.shape)
print(labels.shape)

(22787, 100, 100, 3)
(22787,)


In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [30]:
# 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 데이터 스케일링 (0~1 범위로)
X_train = X_train.astype('float32') / 255
X_val = X_val.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# 클래스 수 계산 (예: 이진 분류)
num_classes = len(np.unique(labels))

# CNN 모델 정의
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
epochs = 10
batch_size = 32

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_val, y_val))

Epoch 1/10
570/570 [==============================] - 28s 48ms/step - loss: 0.7184 - accuracy: 0.5504 - val_loss: 0.6797 - val_accuracy: 0.5472
Epoch 2/10
570/570 [==============================] - 27s 47ms/step - loss: 0.6706 - accuracy: 0.5710 - val_loss: 0.6800 - val_accuracy: 0.5577
Epoch 3/10
570/570 [==============================] - 27s 47ms/step - loss: 0.6617 - accuracy: 0.5905 - val_loss: 0.6844 - val_accuracy: 0.5450
Epoch 4/10
570/570 [==============================] - 27s 47ms/step - loss: 0.6529 - accuracy: 0.6037 - val_loss: 0.6876 - val_accuracy: 0.5410
Epoch 5/10
570/570 [==============================] - 27s 47ms/step - loss: 0.6455 - accuracy: 0.6180 - val_loss: 0.7021 - val_accuracy: 0.5489
Epoch 6/10
570/570 [==============================] - 27s 48ms/step - loss: 0.6396 - accuracy: 0.6266 - val_loss: 0.6951 - val_accuracy: 0.5344
Epoch 7/10
570/570 [==============================] - 27s 47ms/step - loss: 0.6322 - accuracy: 0.6337 - val_loss: 0.7047 - val_accuracy: